# Нейронная сеть Хопфилда

Импортируем инструменты

In [1]:
import numpy as np
from src import HopfieldNetwork

from PIL import Image

Создадим обучающую выборку эталонных объектов

In [2]:
np.random.seed(42)
sample = np.array(np.sign(np.random.random(size=(3, 8)) - 0.5), dtype=np.int32)

print(sample)

[[-1  1  1  1 -1 -1 -1  1]
 [ 1  1 -1  1  1 -1 -1 -1]
 [-1  1 -1 -1  1 -1 -1 -1]]


Обучим сеть Хопфилда по указанной выборке

In [3]:
hn = HopfieldNetwork(max_iter=30).fit(sample)

Посмотрим на матрицу весов

In [7]:
print(hn.weights.round(decimals=3))

[[ 0.    -0.333 -0.333  0.333  0.333  0.333  0.333 -0.333]
 [-0.333  0.    -0.333  0.333  0.333 -1.    -1.    -0.333]
 [-0.333 -0.333  0.     0.333 -1.     0.333  0.333  1.   ]
 [ 0.333  0.333  0.333  0.    -0.333 -0.333 -0.333  0.333]
 [ 0.333  0.333 -1.    -0.333  0.    -0.333 -0.333 -1.   ]
 [ 0.333 -1.     0.333 -0.333 -0.333  0.     1.     0.333]
 [ 0.333 -1.     0.333 -0.333 -0.333  1.     0.     0.333]
 [-0.333 -0.333  1.     0.333 -1.     0.333  0.333  0.   ]]


Проверим, что предсказания для эталонных объектов – сами объекты

In [8]:
np.all(hn.predict(sample) == sample)

True

Возмьмём какую нибудь другую выборку и выполним предсказания для неё

In [16]:
np.random.seed(45)
test = np.array(np.sign(np.random.random(size=(3, 8)) - 0.5), dtype=np.int32)

print(sample, test == sample, sep="\n" * 2)

[[-1  1  1  1 -1 -1 -1  1]
 [ 1  1 -1  1  1 -1 -1 -1]
 [-1  1 -1 -1  1 -1 -1 -1]]

[[False  True False False  True  True  True False]
 [False  True False  True  True  True False  True]
 [False  True  True  True  True  True  True  True]]


In [17]:
pred = hn.predict(test)

print(pred)

[[ 1  1 -1 -1  1 -1 -1 -1]
 [ 1  1 -1 -1  1 -1 -1 -1]
 [ 1  1 -1 -1  1 -1 -1 -1]]


Проверим, что результатом предсказания оказались эталонные объекты

In [18]:
all(instance in sample for instance in pred)

True

Импортируем датасет изображений